# Structure of this Notebook:
For the analysis, the data regarding actual and forecast wind feed-in was downloaded from the information platform of the four German transmission system operators (TSO). In Excel, the actual and forecast wind feed-in was combined into an Excel spreadsheet, broken down by the four TSO. Also irrelevant columns like e.g. time to, were removed. 

In Section 1 the required libraries are imported. 


In the sections 2 to 5 the wind data from the four control area are prepared for the analysis. 
The following steps each relate to the data sets of the four TSOs:In this context. 
In section 6 the four data sets are combined into one data set and subjected to further processing. cd

# 1. Importing the required libaries

In [293]:
# Importing the required libaries
import pandas as pd
import numpy as np
import seaborn as sns #visualization
import matplotlib.pyplot as plt #visualization
from datetime import datetime , timedelta
import sqlalchemy
import psycopg2
from sql import engine

# 2. Control area: 50Hertz

## 2.1 Loading the data into data frames

In [294]:
wind_50Hertz= pd.read_excel("Data/wind_raw_data.xlsx", sheet_name = '50Hertz', header = 0, parse_dates= [['date', 'time']])

## 2.2 Checking the types of the data and count of observations

In [295]:
wind_50Hertz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315648 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315648 non-null  datetime64[ns]
 1   timezone   315648 non-null  object        
 2   pred       315064 non-null  float64       
 3   act        315585 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 9.6+ MB


## 2.3 Checking and removing duplicated rows regarding the time change

In [296]:
wind_50Hertz[wind_50Hertz.duplicated(['date_time'], keep = False)]

,date_time,timezone,pred,act
28900,2012-10-28 02:00:00,CEST,457.11,345.00
28901,2012-10-28 02:15:00,CEST,437.22,347.50
28902,2012-10-28 02:30:00,CEST,427.33,350.00
28903,2012-10-28 02:45:00,CEST,417.44,352.50
28904,2012-10-28 02:00:00,CET,407.56,355.00
...,...,...,...,...
309127,2020-10-25 02:45:00,CEST,219.09,238.68
309128,2020-10-25 02:00:00,CET,220.56,242.86
309129,2020-10-25 02:15:00,CET,222.21,242.04
309130,2020-10-25 02:30:00,CET,223.94,240.07


In [297]:
wind_50Hertz.drop_duplicates(['date_time'], keep = False, inplace = True)

## 2.4 Checking and handling missing values

In [298]:
print(wind_50Hertz[wind_50Hertz.isnull().any(axis=1)])

                 date_time timezone    pred    act
172032 2016-11-27 00:00:00      CET     NaN  289.0
172033 2016-11-27 00:15:00      CET     NaN  295.0
172034 2016-11-27 00:30:00      CET     NaN  300.0
172035 2016-11-27 00:45:00      CET     NaN  315.0
172036 2016-11-27 01:00:00      CET     NaN  318.0
...                    ...      ...     ...    ...
311991 2020-11-23 21:45:00      CET  130.69    NaN
311993 2020-11-23 22:15:00      CET  124.16    NaN
311995 2020-11-23 22:45:00      CET  118.85    NaN
311997 2020-11-23 23:15:00      CET  114.05    NaN
311999 2020-11-23 23:45:00      CET  109.61    NaN

[647 rows x 4 columns]


In [299]:
#Print the number of missing values per columns
print(wind_50Hertz.isnull().sum(),"\n")

date_time      0
timezone       0
pred         584
act           63
dtype: int64 



In [300]:
wind_50Hertz['pred'].replace(to_replace = np.nan, method = 'ffill', inplace = True)
wind_50Hertz['act'].replace(to_replace = np.nan, method = 'ffill', inplace = True)

## 2.5 Transforming the data set in hourly

In [301]:
wind_50Hertz = wind_50Hertz.set_index('date_time')
wind_50Hertz = wind_50Hertz.resample('H').sum()

In [302]:
wind_50Hertz.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78912 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Freq: H
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pred    78912 non-null  float64
 1   act     78912 non-null  float64
dtypes: float64(2)
memory usage: 1.8 MB


## 2.6 Adding the controlzone

In [303]:
wind_50Hertz['control_zone'] = '50Hertz'
wind_amprion.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78912 entries, 2012-01-01 00:00:00 to 2020-12-31 23:00:00
Freq: H
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pred          78912 non-null  float64
 1   act           78912 non-null  float64
 2   control_zone  78912 non-null  object 
dtypes: float64(2), object(1)
memory usage: 2.4+ MB


# 3. Control area: Amprion

## 3.1 Loading the data into a data frame

In [304]:
wind_amprion= pd.read_excel("Data/wind_raw_data.xlsx", sheet_name = '50Hertz', header = 0, parse_dates= [['date', 'time']])

## 3.2 Checking the types of the data and count of observations

In [305]:
wind_amprion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315648 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315648 non-null  datetime64[ns]
 1   timezone   315648 non-null  object        
 2   pred       315064 non-null  float64       
 3   act        315585 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 9.6+ MB


## 3.3 Checking and removing duplicated rows regarding the time change

In [306]:
print(wind_amprion[wind_amprion.duplicated(['date_time'], keep = False)])

                 date_time timezone    pred     act
28900  2012-10-28 02:00:00     CEST  457.11  345.00
28901  2012-10-28 02:15:00     CEST  437.22  347.50
28902  2012-10-28 02:30:00     CEST  427.33  350.00
28903  2012-10-28 02:45:00     CEST  417.44  352.50
28904  2012-10-28 02:00:00      CET  407.56  355.00
...                    ...      ...     ...     ...
309127 2020-10-25 02:45:00     CEST  219.09  238.68
309128 2020-10-25 02:00:00      CET  220.56  242.86
309129 2020-10-25 02:15:00      CET  222.21  242.04
309130 2020-10-25 02:30:00      CET  223.94  240.07
309131 2020-10-25 02:45:00      CET  225.84  238.07

[72 rows x 4 columns]


In [307]:
wind_amprion.drop_duplicates(['date_time'], keep = False, inplace = True)

## 3.4 Checking and handling missing values

In [308]:
#Print the number of missing values per columns
print(wind_amprion.isnull().sum(),"\n")

date_time      0
timezone       0
pred         584
act           63
dtype: int64 



In [309]:
wind_amprion['pred'].replace(to_replace = np.nan, method = 'ffill', inplace = True)
wind_amprion['act'].replace(to_replace = np.nan, method = 'ffill', inplace = True)

In [310]:
wind_amprion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315576 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315576 non-null  datetime64[ns]
 1   timezone   315576 non-null  object        
 2   pred       315576 non-null  float64       
 3   act        315576 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 12.0+ MB


## 3.5 Transforming the data set in hourly

In [311]:
wind_amprion = wind_amprion.set_index('date_time')
wind_amprion = wind_amprion.resample('H').sum()

In [312]:
wind_amprion.head()

,pred,act
date_time,,
2012-01-01 00:00:00,4344.0,3483.0
2012-01-01 01:00:00,4496.0,3319.0
2012-01-01 02:00:00,4765.0,3319.0
2012-01-01 03:00:00,5004.0,3575.0
2012-01-01 04:00:00,5321.0,3820.0


## 3.6 Adding the control zone

In [313]:
wind_amprion['control_zone'] = 'Amprion'
wind_amprion.head()

,pred,act,control_zone
date_time,,,
2012-01-01 00:00:00,4344.0,3483.0,Amprion
2012-01-01 01:00:00,4496.0,3319.0,Amprion
2012-01-01 02:00:00,4765.0,3319.0,Amprion
2012-01-01 03:00:00,5004.0,3575.0,Amprion
2012-01-01 04:00:00,5321.0,3820.0,Amprion


# 4. Control zone: Tennet

## 4.1 Loading the data into a data frame

In [314]:
wind_tennet= pd.read_excel("Data/wind_raw_data.xlsx", sheet_name = 'Tennet', header = 0, parse_dates= [['date', 'time']])

## 4.2 Checking the types of the data and count of observations

In [315]:
wind_tennet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315648 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315648 non-null  datetime64[ns]
 1   timezone   315648 non-null  object        
 2   pred       315648 non-null  float64       
 3   act        315648 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 9.6+ MB


## 4.3 Checking and removing duplicated rows regarding the time change

In [316]:
print(wind_tennet[wind_tennet.duplicated(['date_time'], keep = False)])

                 date_time timezone    pred     act
28900  2012-10-28 02:00:00      CEST  184.8  227.23
28901  2012-10-28 02:15:00      CEST  176.0  221.72
28902  2012-10-28 02:30:00      CEST  167.2  217.19
28903  2012-10-28 02:45:00      CEST  157.6  205.00
28904  2012-10-28 02:00:00       CET  130.4  205.00
...                    ...       ...    ...     ...
309127 2020-10-25 02:45:00      CEST  299.0  266.67
309128 2020-10-25 02:00:00       CET  301.0  273.23
309129 2020-10-25 02:15:00       CET  303.0  276.34
309130 2020-10-25 02:30:00       CET  304.0  276.70
309131 2020-10-25 02:45:00       CET  307.0  272.59

[72 rows x 4 columns]


In [317]:
wind_tennet.drop_duplicates(['date_time'], keep = False, inplace = True)

## 4.4 Checking and handling missing values

In [318]:
#Print the number of missing values per columns
print(wind_tennet.isnull().sum(),"\n")

date_time    0
timezone     0
pred         0
act          0
dtype: int64 



In [319]:
wind_tennet['pred'].replace(to_replace = np.nan, method = 'ffill', inplace = True)
wind_tennet['act'].replace(to_replace = np.nan, method = 'ffill', inplace = True)

In [320]:
wind_tennet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315576 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315576 non-null  datetime64[ns]
 1   timezone   315576 non-null  object        
 2   pred       315576 non-null  float64       
 3   act        315576 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 12.0+ MB


## 4.5 Transforming the data set in hourly

In [321]:
wind_tennet = wind_tennet.set_index('date_time')
wind_tennet = wind_tennet.resample('H').sum()

In [322]:
wind_tennet.head()

,pred,act
date_time,,
2012-01-01 00:00:00,2943.08,3101.94
2012-01-01 01:00:00,3310.20,3499.41
2012-01-01 02:00:00,3714.40,4342.17
2012-01-01 03:00:00,4132.28,5288.10
2012-01-01 04:00:00,4566.36,5774.19


## 4.6 Adding the control zone

In [323]:
wind_tennet['control_zone'] = 'Tennet'
wind_tennet.head()

,pred,act,control_zone
date_time,,,
2012-01-01 00:00:00,2943.08,3101.94,Tennet
2012-01-01 01:00:00,3310.20,3499.41,Tennet
2012-01-01 02:00:00,3714.40,4342.17,Tennet
2012-01-01 03:00:00,4132.28,5288.10,Tennet
2012-01-01 04:00:00,4566.36,5774.19,Tennet


# 5. Control area: TransnetBW

## 5.1 Loading the data into a data frame

In [324]:
wind_transnetbw= pd.read_excel("Data/wind_raw_data.xlsx", sheet_name = 'TransnetBW', header = 0, parse_dates= [['date', 'time']])

## 3.2 Checking the types of the data and count of observations

In [325]:
wind_transnetbw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315648 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315648 non-null  datetime64[ns]
 1   timezone   315648 non-null  object        
 2   pred       315246 non-null  float64       
 3   act        315328 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 9.6+ MB


## 3.3 Checking and removing duplicated rows regarding the time change

In [326]:
print(wind_transnetbw[wind_transnetbw.duplicated(['date_time'], keep = False)])

                 date_time timezone   pred    act
28900  2012-10-28 02:00:00      CEST   NaN    NaN
28901  2012-10-28 02:15:00      CEST   NaN    NaN
28902  2012-10-28 02:30:00      CEST   NaN    NaN
28903  2012-10-28 02:45:00      CEST   NaN    NaN
28904  2012-10-28 02:00:00       CET   NaN    NaN
...                    ...       ...   ...    ...
309127 2020-10-25 02:45:00      CEST  26.0  24.86
309128 2020-10-25 02:00:00       CET  26.0  24.33
309129 2020-10-25 02:15:00       CET  27.0  24.14
309130 2020-10-25 02:30:00       CET  27.0  26.44
309131 2020-10-25 02:45:00       CET  28.0  27.25

[72 rows x 4 columns]


In [327]:
wind_transnetbw.drop_duplicates(['date_time'], keep = False, inplace = True)

## 3.4 Checking and handling missing values

In [328]:
#Print the number of missing values per columns
print(wind_transnetbw.isnull().sum(),"\n")

date_time      0
timezone       0
pred         386
act          312
dtype: int64 



In [329]:
wind_transnetbw['pred'].replace(to_replace = np.nan, method = 'ffill', inplace = True)
wind_transnetbw['act'].replace(to_replace = np.nan, method = 'ffill', inplace = True)

In [330]:
wind_transnetbw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315576 entries, 0 to 315647
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_time  315576 non-null  datetime64[ns]
 1   timezone   315576 non-null  object        
 2   pred       315576 non-null  float64       
 3   act        315576 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 12.0+ MB


## 3.5 Transforming the data set in hourly

In [331]:
wind_transnetbw = wind_transnetbw.set_index('date_time')
wind_transnetbw = wind_transnetbw.resample('H').sum()

In [332]:
wind_transnetbw.head()

,pred,act
date_time,,
2012-01-01 00:00:00,348.0,716.2
2012-01-01 01:00:00,274.0,590.7
2012-01-01 02:00:00,293.0,431.3
2012-01-01 03:00:00,326.0,533.7
2012-01-01 04:00:00,338.0,615.3


## 3.6 Adding the control zone

In [333]:
wind_transnetbw['control_zone'] = 'TransnetBW'
wind_transnetbw.head()

,pred,act,control_zone
date_time,,,
2012-01-01 00:00:00,348.0,716.2,TransnetBW
2012-01-01 01:00:00,274.0,590.7,TransnetBW
2012-01-01 02:00:00,293.0,431.3,TransnetBW
2012-01-01 03:00:00,326.0,533.7,TransnetBW
2012-01-01 04:00:00,338.0,615.3,TransnetBW


# 6. Final Data Set

## 6.1 Combining the DataFrames

In [334]:
wind = pd.concat([wind_50Hertz, wind_amprion, wind_tennet, wind_transnetbw], axis = 0)

## 6.2 Adding more Features

### 6.2.1 Forecast Error

In [335]:
wind['delta'] = wind['pred'] - wind['act']

### 6.2.1 Year

In [336]:
wind['year'] = wind.index.year
wind.head()

,pred,act,control_zone,delta,year
date_time,,,,,
2012-01-01 00:00:00,4344.0,3483.0,50Hertz,861.0,2012
2012-01-01 01:00:00,4496.0,3319.0,50Hertz,1177.0,2012
2012-01-01 02:00:00,4765.0,3319.0,50Hertz,1446.0,2012
2012-01-01 03:00:00,5004.0,3575.0,50Hertz,1429.0,2012
2012-01-01 04:00:00,5321.0,3820.0,50Hertz,1501.0,2012


### 6.2.2 Month

In [337]:
wind['month'] = wind.index.month

### 6.2.3 Hour

In [338]:
wind['hour'] = wind.index.hour

In [339]:
wind.head()

,pred,act,control_zone,delta,year,month,hour
date_time,,,,,,,
2012-01-01 00:00:00,4344.0,3483.0,50Hertz,861.0,2012,1,0
2012-01-01 01:00:00,4496.0,3319.0,50Hertz,1177.0,2012,1,1
2012-01-01 02:00:00,4765.0,3319.0,50Hertz,1446.0,2012,1,2
2012-01-01 03:00:00,5004.0,3575.0,50Hertz,1429.0,2012,1,3
2012-01-01 04:00:00,5321.0,3820.0,50Hertz,1501.0,2012,1,4


### 6.2.4 Resetting the Datetime Index

In [340]:
wind = wind.reset_index(drop=False)
wind.head()

,date_time,pred,act,control_zone,delta,year,month,hour
0,2012-01-01 00:00:00,4344.0,3483.0,50Hertz,861.0,2012,1,0
1,2012-01-01 01:00:00,4496.0,3319.0,50Hertz,1177.0,2012,1,1
2,2012-01-01 02:00:00,4765.0,3319.0,50Hertz,1446.0,2012,1,2
3,2012-01-01 03:00:00,5004.0,3575.0,50Hertz,1429.0,2012,1,3
4,2012-01-01 04:00:00,5321.0,3820.0,50Hertz,1501.0,2012,1,4


# 7. Pushing the the prepared data to a table in the database

In [342]:
table = 'wind_LC'
wind.to_sql(table, engine, index=False, if_exists="replace", 
    method='multi', chunksize=5000)
print('Successfully pushed!')

Successfully pushed!
